# 1. Import bibliotek

In [1]:
import requests
from bs4 import BeautifulSoup
import re

D:\Users\andrzej_jedrzejowski\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# 2. Definicje

In [2]:
#Lista wariantów
property_t_list = ["mieszkanie", "dom", "pokoj", "dzialka", "lokal", "haleimagazyny", "garaz"]
#usunięty property_t "inwestycja" - to jest dziwne
transaction_list = ["sprzedaz", "wynajem"]

In [3]:
#Słowniki wariantów - każdy słownik zawiera listę zmiennych dopuszczonych dla danego wariantu
dic_variant = {}

dic_variant ["mieszkanie_sprzedaz"] = ["localization", "distanceRadius", "market", "ownerType",
                                       "floorsNumberMin", "floorsNumberMax", "floor", "roomsNumber", "areaMin", "areaMax",
                                       "buildingType", "buildYearMin", "buildYearMax",
                                       "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                       "daysSinceCreated", "extras"]

dic_variant["mieszkanie_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner", 
                                     "floorsNumberMin", "floorsNumberMax", "floor", "roomsNumber", "areaMin", "areaMax",
                                     "buildingType", "buildYearMin", "buildYearMax",
                                     "priceMin", "priceMax",
                                     "daysSinceCreated", "extras", "heating", "media", "freeFrom", "isForStudents"]

dic_variant ["dom_sprzedaz"] = ["localization", "distanceRadius", "market", "ownerType",
                                "roomsNumber", "areaMin", "areaMax", "terrainAreaMin", "terrainAreaMax",
                                "buildingType", "buildYearMin", "buildYearMax", "buildingMaterial", "roofType",
                                "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                "daysSinceCreated", "isRecreational","isBungalov"]

dic_variant ["dom_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner",
                               "roomsNumber", "areaMin", "areaMax", "terrainAreaMin", "terrainAreaMax",
                               "buildingType", "buildYearMin", "buildYearMax",
                               "priceMin", "priceMax",
                               "daysSinceCreated", "freeFrom", "isRecreational", "media"]

dic_variant ["pokoj_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner",
                                 "peoplePerRoom",
                                 "priceMin", "priceMax",
                                 "daysSinceCreated", "media", "freeFrom"]

dic_variant ["dzialka_sprzedaz"] = ["localization", "distanceRadius", "isPrivateOwner", 
                                    "areaMin", "areaMax",
                                    "plotType",
                                    "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                    "daysSinceCreated", "vicinity"]

dic_variant ["dzialka_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner", 
                                    "areaMin", "areaMax",
                                    "plotType",
                                    "priceMin", "priceMax",
                                    "daysSinceCreated", "freeFrom"]

dic_variant ["lokal_sprzedaz"] = ["localization", "distanceRadius", "market", "ownerType",
                                  "floor", "roomsNumber", "areaMin", "areaMax",
                                  "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                  "daysSinceCreated", "extras", "useTypes"]

dic_variant ["lokal_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner", 
                                  "floor", "areaMin", "areaMax",
                                  "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                  "daysSinceCreated", "freeFrom", "extras", "useTypes"]

dic_variant ["haleimagazyny_sprzedaz"] = ["localization", "distanceRadius", "market", "ownerType",
                                          "roomsNumber", "areaMin", "areaMax", "heightMin", "heightMax",
                                          "priceMin", "priceMax", "pricePerMeterMin", "pricePerMeterMax",
                                          "structure","heating",
                                          "daysSinceCreated", "useTypes"]

dic_variant ["haleimagazyny_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner",
                                          "areaMin", "areaMax", "heightMin", "heightMax",
                                          "priceMin", "priceMax", 
                                          "structure","heating",
                                          "daysSinceCreated", "freeFrom", "useTypes"]

dic_variant ["garaz_sprzedaz"] = ["localization", "distanceRadius", "market", "ownerType",
                                  "roomsNumber", "areaMin", "areaMax",
                                  "priceMin", "priceMax",
                                  "daysSinceCreated"]

dic_variant ["garaz_wynajem"] = ["localization", "distanceRadius", "isPrivateOwner",
                                 "areaMin", "areaMax",
                                 "priceMin", "priceMax",
                                 "daysSinceCreated", "freeFrom",]

In [4]:
#Lista unikatowych zmiennych dostępnych w sumie wszystkich wariantów
variable_list = []
for key in dic_variant:
    for variable in dic_variant[key]:
        if variable not in variable_list: variable_list.append(variable)
variable_list.sort()

# for variable in variable_list:
#     print(variable)

In [5]:
#Słowniki wariantów - każdy słownik zawiera listę wartości dopuszczonych dla danej unikatowej zmiennej
dic_value = {}

dic_value ["areaMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                         "type": "number",
                         "character": "common",
                         "value": "Brak określonej wartości"}
dic_value ["areaMax"] = {"description":"Dowolna liczba całkowita większa od 0 oraz większa od areaMin (jeżeli podano)",
                         "type": "numberMax",
                         "character": "common",
                         "value": "Brak określonej wartości"}
dic_value ["buildYearMin"] = {"description":"Dowolna liczba całkowita z przedziału 1000 - 2100",
                              "type": "year",
                              "character": "common",
                              "value": "Brak określonej wartości"}
dic_value ["buildYearMax"] = {"description":"Dowolna liczba całkowita z przedziału 1000 - 2100 oraz większa od bulidYearMin (jeżeli podano)",
                              "type": "yearMax",
                              "character": "common",
                              "value": "Brak określonej wartości"}
dic_value ["buildingMaterial"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                                  "type": "list",
                                  "character": "common",
                                  "value": ["BRICK", "WOOD", "BREEZEBLOCK", "HYDROTON", "CONCRETE_PLATE", "CONCRETE", "SILIKAT", "CELLULAR_CONCRETE", "OTHER"]}
dic_value ["buildingType"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                              "type": "list",
                              "character": "specific",
                              "value": {"mieszkanie_sprzedaz": ["BLOCK","TENEMENT", "HOUSE", "INFILL", "RIBBON", "APARTMENT", "LOFT"],
                                        "mieszkanie_wynajem": ["BLOCK","TENEMENT", "HOUSE", "INFILL", "RIBBON", "APARTMENT", "LOFT"],
                                        "dom_sprzedaz": ["DETACHED", "SEMI_DETACHED", "RIBBON", "TENEMENT", "RESIDENCE", "FARM"],
                                        "dom_wynajem": ["DETACHED", "SEMI_DETACHED", "RIBBON", "TENEMENT", "RESIDENCE", "FARM"]}}
dic_value ["daysSinceCreated"] = {"description": "Jedna wartość z listy 'value'",
                                  "type": "list",
                                  "character": "common",
                                  "value": ["1", "3", "7", "14"]}
dic_value ["distanceRadius"] = {"description": "Jedna wartość z listy 'value'",
                                "type": "list",
                                "character": "common",
                                "value": ["0", "5", "10", "15", "25", "50", "75"]}
dic_value ["extras"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                        "type": "list",
                        "character": "specific",
                        "value": {"mieszkanie_sprzedaz": ["AIR_CONDITIONING", "BALCONY", "BASEMENT", "GARAGE", "GARDEN", "LIFT", "NON_SMOKERS_ONLY", "SEPARATE_KITCHEN", "TERRACE", "TWO_STOREY", "USABLE_ROOM"],
                                  "mieszkanie_wynajem": ["AIR_CONDITIONING", "BALCONY", "BASEMENT", "GARAGE", "GARDEN", "LIFT", "NON_SMOKERS_ONLY", "SEPARATE_KITCHEN", "TERRACE", "TWO_STOREY", "USABLE_ROOM"],
                                  "lokal_sprzedaz": ["AIR_CONDITIONING", "ASPHALT_ACCESS", "ELEVATOR", "FURNITURE", "HEATING", "PARKING", "SHOP_WINDOW"],
                                  "lokal_wynajem": ["AIR_CONDITIONING", "ASPHALT_ACCESS", "ELEVATOR", "FURNITURE", "HEATING", "PARKING", "SHOP_WINDOW"]}}
dic_value ["floor"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                       "type": "list",
                       "character": "common",
                       "value": ["CELLAR","GROUND", "FIRST", "SECOND", "THIRD", "FOURTH", "FIFTH", "SIXTH", "SEVENTH", "EIGHTH", "NINTH", "TENTH", "ABOVE_TENTH", "GARRET"]}
dic_value ["floorsNumberMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                                 "type": "number",
                                 "character": "common",
                                 "value": "Brak określonej wartości"}
dic_value ["floorsNumberMax"] = {"description":"Dowolna liczba całkowita większa od 0 lub większa od floorsNumberMin (jeżeli podano)",
                                 "type": "numberMax",
                                 "character": "common",
                                 "value": "Brak określonej wartości"}
dic_value ["freeFrom"] = {"description": "Jedna wartość z listy 'value'",
                          "type": "list",
                          "character": "common",
                          "value": ["NOW","THIRTY_DAYS", "NINETY_DAYS"]}
dic_value ["heating"] = {"description":{"mieszkanie_wynajem":"Jedna lub więcej wartości z listy 'value'",
                                        "haleimagazyny_sprzedaz": "Jedna wartość z listy 'value'",
                                        "haleimagazyny_wynajem": "Jedna wartość z listy 'value'"},
                         "type": "list",
                         "character": "specific",
                         "value": {"mieszkanie_wynajem":["URBAN", "GAS", "TILED_STOVE", "ELECTRICAL", "BOILER_ROOM", "OTHER"],
                                   "haleimagazyny_sprzedaz": ["YES", "NO"],
                                   "haleimagazyny_wynajem": ["YES", "NO"]}}                      
dic_value ["heightMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                           "type": "number",
                           "character": "common",
                           "value": "Brak określonej wartości"}
dic_value ["heightMax"] = {"description":"Dowolna liczba całkowita większa od 0 lub większa od heightMin (jeżeli podano)",
                            "type": "numberMax",
                            "character": "common",
                            "value": "Brak określonej wartości"}
dic_value ["isBungalov"] = {"description": "Jedna wartość z listy 'value'",
                            "type": "list",
                            "character": "common",
                            "value": ["true"]}
dic_value ["isForStudents"] = {"description": "Jedna wartość z listy 'value'",
                               "type": "list",
                               "character": "common",
                               "value": ["true"]}
dic_value ["isPrivateOwner"] = {"description": "Jedna wartość z listy 'value'",
                                "type": "list",
                                "character": "common",
                                "value": ["true"]}
dic_value ["isRecreational"] = {"description": "Jedna wartość z listy 'value'",
                                "type": "list",
                                "character": "common",
                                "value": ["true"]}
dic_value ["localization"] = {"description":"Dowolny tekst",
                              "type": "text",
                              "character": "common",
                              "value": "Brak określonej wartości"}
dic_value ["market"] = {"description": "Jedna wartość z listy 'value'",
                        "type": "list",
                        "character": "common",
                        "value": ["ALL", "PRIMARY", "SECONDARY"]}
dic_value ["media"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                       "type": "list",
                       "character": "specific",
                       "value": {"mieszkanie_wynajem":["INTERNET","CABLE_TELEVISION","PHONE"],
                                 "dom_wynajem":["CABLE_TELEVISION","CESSPOOL","ELECTRICITY","GAS","INTERNET","PHONE","SEWAGE","WATER","WATER_PURIFICATION"],
                                 "pokoj_wynajem":["INTERNET","CABLE_TELEVISION","PHONE"]}}
dic_value ["ownerType"] = {"description": "Jedna wartość z listy 'value'",
                           "type": "list",
                           "character": "common",
                           "value": ["DEVELOPER","AGENCY", "PRIVATE"]}
dic_value ["peoplePerRoom"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                               "type": "list",
                               "character": "common",
                               "value": ["ONE", "TWO", "THREE"]}
dic_value ["plotType"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                          "type": "list",
                          "character": "common",
                          "value": ["BUILDING", "AGRICULTURAL", "RECREATIONAL", "COMMERCIAL", "WOODLAND", "AGRICULTURAL_BUILDING", "HABITAT", "COTHER"]}
dic_value ["priceMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                          "type": "number",
                          "character": "common",
                          "value": "Brak określonej wartości"}
dic_value ["priceMax"] = {"description":"Dowolna liczba całkowita większa od 0 lub większa od priceMin (jeżeli podano)",
                          "type": "numberMax",
                          "character": "common",
                          "value": "Brak określonej wartości"}
dic_value ["pricePerMeterMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                                  "type": "number",
                                  "character": "common",
                                  "value": "Brak określonej wartości"}
dic_value ["pricePerMeterMax"] = {"description":"Dowolna liczba całkowita większa od 0 lub większa od pricePerMeterMin (jeżeli podano)",
                                  "type": "numberMax",
                                  "character": "common",
                                  "value": "Brak określonej wartości"}
dic_value ["roofType"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                          "type": "list",
                          "character": "common",
                          "value": ["NOTANY", "FLAT", "DIAGONAL"]}
dic_value ["roomsNumber"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                             "type": "list",
                             "character": "common",
                             "value": ["ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN", "EIGHT", "NINE", "TEN", "MORE"]}
dic_value ["structure"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                           "type": "list",
                           "character": "common",
                           "value": ["STEEL", "BRICK", "SHED", "TENT", "WOOD", "GLASS"]}
dic_value ["terrainAreaMin"] = {"description":"Dowolna liczba całkowita większa od 0",
                                "type": "number",
                                "character": "common",
                                "value": "Brak określonej wartości"}
dic_value ["terrainAreaMax"] = {"description":"Dowolna liczba całkowita większa od 0 lub większa od terrainAreaMin (jeżeli podano)",
                                "type": "numberMax",
                                "character": "common",
                                "value": "Brak określonej wartości"}
dic_value ["useTypes"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                          "type": "list",
                          "character": "specific",
                          "value": {"lokal_sprzedaz": ["GASTRONOMY", "HOTEL", "INDUSTRIAL", "OFFICE", "RETAIL", "SERVICES"],
                                    "lokal_wynajem": ["GASTRONOMY", "HOTEL", "INDUSTRIAL", "OFFICE", "RETAIL", "SERVICES"],
                                    "haleimagazyny_sprzedaz": ["STOCK", "MANUFACTURING", "OFFICE", "COMMERCIAL"],
                                    "haleimagazyny_wynajem":["STOCK", "MANUFACTURING", "OFFICE", "COMMERCIAL"]}}
dic_value ["vicinity"] = {"description":"Jedna lub więcej wartości z listy 'value'",
                          "type": "list",
                          "character": "common",
                          "value": ["FOREST", "LAKE", "OPEN_TERRAIN", "MOUNTAINS", "SEA"]}

In [6]:
dic_value ["buildingType"]["value"]["mieszkanie_sprzedaz"]

['BLOCK', 'TENEMENT', 'HOUSE', 'INFILL', 'RIBBON', 'APARTMENT', 'LOFT']

# 3. Funkcje

In [8]:
#Funkcja variant_type generuje nazwę wariantu
def variant_type (property_t, transaction):
    if property_t in property_t_list:
        if transaction in transaction_list:
            if not (property_t == "pokoj" and transaction == "sprzedaz"):
                variant = property_t+"_"+transaction
            else:
                print (f'Variant pokoj_sprzedaz is not avaliable')
                variant = ""
        else: 
            print (f'Wrong transaction value (second argument). Choose from list: {transaction_list}')
            variant = ""
    else: 
        print (f'Wrong property_t value (first argument). Choose from list: {property_t_list}')
        variant = ""
    
    return variant

In [10]:
# variant_type ("mieszkanie", "sprzedaz")

In [11]:
#Funkcja generuje informację o listach argumentów dostępnych dla argumentów property_t oraz transaction
def show_variant_avaliable ():
    print(f'Dla argumentu property_t dopuszcza się argumenty z listy: {property_t_list}')
    print(f'Dla argumentu transaction dopuszcza się argumenty z listy: {transaction_list}')
    print("Jedynym nie dopuszczonym połączeniem jest 'pokój' + 'sprzedaz'")

In [13]:
# show_variant_avaliable ()

In [15]:
#Funkcja generuje listę zmiennych dostępnych dla variantu generowanego na podstawie argumentów property_t oraz transaction
def show_variable_avaliable (property_t, transaction):
    variant = variant_type (property_t, transaction)
    if len (variant) > 0:
        variable_list = dic_variant[variant]
        return variable_list

In [18]:
# show_variable_avaliable("mieszkanie","wynajem")

In [19]:
#Funkcja generuje informacje o wartościac dostępnych dla danej zmiennej
def show_value_avaliable (variable):
    if variable in variable_list:
        for key in dic_value[variable]:
            if key != "value":
                print(f'{key}:{dic_value[variable][key]}')
            else:
                if dic_value[variable]['character'] != "specific":
                    print(f'{key}:{dic_value[variable][key]}')
                else:
                    for key_2 in dic_value[variable][key]:
                        print(f'{key}: {key_2}:{dic_value[variable][key][key_2]}')
    else:
        print(f"Nie ma takiej zmiennej. Wybierz zmienną z listy:")
        for element in variable_list: print (element)

In [21]:
# show_value_avaliable('extras')

In [25]:
#Funkcja generuje pełen słownik zzmiennych i wartości dostępnych dla wariantu enerowanego na podstawie argumentów property_t oraz transaction
def get_variant_dic (property_t, transaction):
    variant = variant_type(property_t, transaction)
    if len (variant) > 0:
        variable_list = show_variable_avaliable (property_t, transaction)
        variant_dic = {}
        for variable in variable_list:
            variant_dic[variable] = {}
            for key in dic_value [variable]:
                if type(dic_value [variable][key]) != dict:
                    variant_dic [variable][key] = dic_value [variable][key]
                else:
                    variant_dic [variable][key] = dic_value [variable][key][variant]
        return variant_dic
    else:
        pass

In [26]:
# get_variant_dic ("dom", "wynajem")

In [27]:
#Funkcja weryfikuje poprawność podanych zmiennych
def variable_verification (property_t, transaction, variables):
    variant = variant_type (property_t, transaction)
    variable_avaliable = show_variable_avaliable (property_t, transaction)
    variable_error = 0
    correct_variable_list = []
    for variable in variables:
        if variable[0] not in variable_avaliable: 
            print (f'Zmienna {variable[0]} jest niedostępna dla wariantu {variant}')
            variable_error +=1
        else:
            correct_variable_list.append(variable[0])
            
    if variable_error > 0: print (f'\nLista dostępnych zmiennych: {variable_avaliable}')
    return correct_variable_list

In [28]:
#Funkcja weryfikuje poprawność podanych wartości
def value_verification (property_t, transaction, variables):
    variant_dic = get_variant_dic (property_t, transaction)
    variable_arguments = []
    error_value = 0
    for variable in variables:
        variable_arguments.append(variable[0])
    for variable in variables:
        if len(variable) != 2:
            print(f'''Zmienna {variable} została podana niepoprawnie.
Należy podać listę z dwoma argumentami [x, y], gdzie x to nazwa zmiennej, a y to jej wartość.
Przykład:["localization", "Warszawa"], ["buildingType", "BLOCK"], ["buildingType", "HOUSE"]\n''')

        else:
            variable_type = variant_dic[variable[0]]["type"]
            values_number = variable_arguments.count(variable[0])              
            n_answers = len (variant_dic[variable[0]]["value"]) if "więcej" in variant_dic[variable[0]]["description"] else 1

            if values_number > n_answers:
                print (f'\nWskazano za dużo wartości dla argumentu {variable[0]}.\nPodano {values_number} wartości. \nLimit to {n_answers} wartości.')
                error_value += 1
                
            if variable_type == "text" and type(variable[1]) != str:
                print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}. \nPodano wartość typu {type(variable[1])}')
                error_value += 1
                
            if "number" in variant_dic[variable[0]]["type"]:
                if type(variable[1]) != int:
                    print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość typu {type(variable[1])}')
                    error_value += 1
                elif variable[1] <1: 
                    print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość mniejszą niż 1')
                    error_value += 1
                elif "Max" in variable [0]:
                    if (variable[0][:-3]+"Min") in variable_arguments and type(variables[variable_arguments.index(variable[0][:-3]+"Min")][1]) == int:
                        if (variable[1] - variables[variable_arguments.index(variable[0][:-3]+"Min")][1]) <= 0:
                            print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość mniejszą niż dla {variables[variable_arguments.index(variable[0][:-3]+"Min")][0]}')
                            error_value += 1
                                
            if "year" in variant_dic[variable[0]]["type"]:
                if type(variable[1]) != int: 
                    print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość typu {type(variable[1])}')
                    error_value += 1
                elif variable[1] <1000 or variable[1] > 2100: 
                    print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość spoza przedziału')
                    error_value += 1
                elif "Max" in variable [0]:
                    if (variable[0][:-3]+"Min") in variable_arguments and type(variables[variable_arguments.index(variable[0][:-3]+"Min")][1]) == int:
                        if (variable[1] - variables[variable_arguments.index(variable[0][:-3]+"Min")][1]) <= 0:
                            print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}.\nPodano wartość nie większą niż dla {variables[variable_arguments.index(variable[0][:-3]+"Min")][0]}')         
                            error_value += 1
                            
            if variable_type == "list" and (type(variable[1]) != str or variable[1] not in variant_dic[variable[0]]["value"]):
                    print(f'\nBłędna wartość dla argumentu {variable[0]}.\nWarunek: {variant_dic[variable[0]]["description"]}. \nPodano wartość {variable[1]} typu {type(variable[1])}. \nWybierz wartość z listy: {variant_dic[variable[0]]["value"]}\n')
                    error_value += 1
    
    return error_value

In [30]:
#Funkcja get_link generuje link
def get_link (property_t, transaction, variables = []):
    
    if len (variant_type (property_t, transaction)) > 0:
        link = f"https://www.otodom.pl/pl/oferty/{transaction}/{property_t}/"
        
        if len(variables) > 0:
            correct_variable_list = variable_verification (property_t, transaction, variables)
            variant_dic = get_variant_dic (property_t, transaction)
            
            if len(correct_variable_list) == len(variables):
                
                if value_verification (property_t, transaction, variables) == 0:

                    if "localization" in correct_variable_list:
                        localization_i = (correct_variable_list.index("localization"))
                        link = link + variables[localization_i][1]
                        correct_variable_list.remove(correct_variable_list[localization_i])

                    correct_variable_list_noduplicat = [] #tworzę listę unikatowych poprawnych zmiennych
                    for cvl in correct_variable_list:
                        if cvl not in correct_variable_list_noduplicat: correct_variable_list_noduplicat.append(cvl)
                    
                    link = link + "?"
                    dic_link = {}
                    for element in correct_variable_list_noduplicat:
                        dic_link[element] = [] #tworzę pusty słownik na bazie unikatowych poprawnych zmiennych, który następnie uzupełniam
                    for element in dic_link:
                        for variable in variables:
                            if variable[0] == element: 
                                if variable[1] not in dic_link[element]: dic_link[element].append(variable[1])

                    for element in dic_link: #sklejam link - w zależności od typu zmiennej, wartości wpisuję w nawiasie kwadratowym lub bez
                        if "więcej" in variant_dic[element]["description"]:
                            link = f"{link}{element}=["
                            for e in dic_link[element]:
                                link = f"{link}{e},"
                            link = f"{link}]&"
                        else:
                            link = f"{link}{element}={dic_link[element][0]}&"
                    
                    print(link)
                      
                else: print(f"\nPROSZĘ POPRAWIĆ ZAPYTANIE")
            else: print(f"\nPROSZĘ POPRAWIĆ ZAPYTANIE")
        else: print(link)
    else: print(f"\nPROSZĘ POPRAWIĆ ZAPYTANIE")

In [44]:
get_link ("mieszkanie", 
          "wynajem",
          [["isPrivateOwner", "true"],
           ["localization", "Warszawa"],
           ["extras", "AIR_CONDITIONING"], 
           ["priceMax", 1300],
           ["extras", "BASEMENT"],
           ["extras", "BALCONY"],
           ["priceMin", 400], 
           ["buildYearMin", 1996], 
           ["extras", "BASEMENT"],
           ["buildYearMax", 2019]]
         )

https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/


In [21]:
# #f_soup pozysukje soup
# def f_soup(link):
#     r = requests.get(link, verify = False)

#     soup = BeautifulSoup(
#         r.text, # kod html strony 
#         'html.parser'  # parser, którego chcemy użyć
#         )
#     return soup

In [22]:
# #f_get_data
# def f_get_data(soup):
#     results = soup.find_all(class_="css-1uevyzq es62z2j29")
#     i=1
#     for result in results:
#         if i <4:
#             i +=1
#         else:
#             try:
#                 title = result.find(class_="css-1873em4 es62z2j24").text
#                 place = result.find(class_="css-80g06k es62z2j23").text
#                 n_rooms = result.find_all(class_="css-348r18 es62z2j20")[0].text.split(" ")[0]
#                 space = result.find_all(class_="css-348r18 es62z2j20")[1].text.split(" ")[0]
#                 space_v = result.find_all(class_="css-348r18 es62z2j20")[1].text.split(" ")[1]
#                 price_m2 = re.findall(
#                     r'\d{1,3}.?\d{3}',
#                     result.find_all(class_="css-348r18 es62z2j20")[2].text)[0].replace("\xa0", "")
#                 price_m2_v = re.findall(
#                     r'\w{1,2}/\w{2}',
#                     result.find_all(class_="css-348r18 es62z2j20")[2].text)[0]
#                 price = re.findall(
#                     r'\d{1,3}.?\d{1,3}.?\d{1,3}',
#                     result.find(class_="css-lk61n3 es62z2j19").text)[0].replace("\xa0", "")
#                 price_v = result.find(class_="css-lk61n3 es62z2j19").text.split('\xa0')[-1]
#                 print(f'{i-3};{title};{place};{price};{price_v};{n_rooms};{space};{space_v};{price_m2};{price_m2_v}')
#                 i+=1
#             except Exception:
#                 pass

In [23]:
# link = f_query("sprzedaz", "mieszkanie", "warszawa")
# soup = f_soup(link)
# f_get_data(soup)